In [1]:
import gym
from gym import spaces
import mujoco_py
import mj_envs
import numpy as np
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback, CallbackList, BaseCallback, CheckpointCallback
from datetime import datetime
import torch
import time


RoboHive:> Registering Arms Envs


/home/carlota/miniconda3/lib/python3.7/site-packages/skvideo/__init__.py:306: UserWarning: ffmpeg/ffprobe not found in path: /home/carlota/miniconda3/lib/python3.7/site-packages/ffmpeg
  warnings.warn("ffmpeg/ffprobe not found in path: " + str(path), UserWarning)


RoboHive:> Registering Myo Envs
RoboHive:> Registering Hand Envs
RoboHive:> Registering Appliances Envs
RoboHive:> Registering Kitchen Envs
RoboHive:> Registering Multi-Task (2 subtasks) Envs
RoboHive:> Registering Multi-Task (9 subtasks) Envs
RoboHive:> Registering TCDM Envs
RoboHive:> Registering Claw Envs


In [2]:
class TensorboardCallback(BaseCallback):
	"""
	Custom callback for plotting additional values in tensorboard.
	"""

	def __init__(self, verbose=0):
	    super(TensorboardCallback, self).__init__(verbose)

	def _on_step(self) -> bool:
	    # Log scalar value (here a random variable)
	    value = self.training_env.get_obs_vec()
	    self.logger.record("obs", value)
	
	    return True


dof_env = ['myoLegReachFixed-v2']

training_steps = 1000000
for env_name in dof_env:
	print('Begin training')
	print(env_name)
	start_time = time.time()
	time_now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
	print(time_now + '\n\n')
	log_path = '/home/carlota/Documents/MyoLeg_script_test/standingBalance/policy_best_model/'+ env_name + '/' + time_now +'/'

	env = gym.make(env_name)

	eval_callback = EvalCallback(env, best_model_save_path=log_path, log_path=log_path, eval_freq=10000, deterministic=True, render=False)

	#policy_kwargs = dict(activation_fn=torch.nn.Sigmoid, net_arch=[dict(pi=[64], vf=[64])])
	policy_kwargs = dict(activation_fn=torch.nn.Sigmoid, net_arch=(dict(pi=[64, 64], vf=[64, 64])))
	model = PPO('MlpPolicy', env, verbose=0, policy_kwargs =policy_kwargs, tensorboard_log="/home/carlota/Documents/MyoLeg_script_test/standingBalance/temp_env_tensorboard/")
	#model = PPO.load('./policy_best_model/'+ env_name +'/2023_03_27_15_32_35/best_model.zip', env, verbose=0, policy_kwargs=policy_kwargs, tensorboard_log="./temp_env_tensorboard/")

	obs_callback = TensorboardCallback()

	callback = CallbackList([obs_callback, eval_callback])

	model.learn(total_timesteps= training_steps, tb_log_name=env_name+"_" + time_now, callback=eval_callback)
	elapsed_time = time.time() - start_time

	hours = int(elapsed_time // 3600)
	minutes = int((elapsed_time % 3600) // 60)
	seconds = int(elapsed_time % 60)

	print(time_now)
	print(f"Elapsed time: {hours} hours, {minutes} minutes, {seconds} seconds.")




Begin training
myoLegReachFixed-v2
2023_05_10_15_06_12


RoboHive:> For environment credits, please cite -
    RoboHive: A unified framework for robot learning | https://sites.google.com/view/robohive
    
Configuring a new session for default_robot(sim)


/home/carlota/miniconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


TypeError: step() takes 2 positional arguments but 3 were given